# Download Package

In [1]:
!pip install simplet5

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.9 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6854 sha256=45d80595ffec01aebf6337c5f3c2ccc19e2c885faf677a547521f3f79130dedb
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a 

# Import Library

In [2]:
# For data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# For model, tokenizer
from simplet5 import SimpleT5
import transformers
from transformers import AutoTokenizer, AutoConfig, T5ForConditionalGeneration

# For neccessary
import os
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load dataset

In [3]:
train_df = pd.read_csv("/kaggle/input/t5-skinchat-data-tokenizer-preparation/data_train.csv")
valid_df = pd.read_csv("/kaggle/input/t5-skinchat-data-tokenizer-preparation/data_valid.csv")
test_df = pd.read_csv("/kaggle/input/t5-skinchat-data-tokenizer-preparation/data_test.csv")

train_df.shape, valid_df.shape, test_df.shape

((220697, 2), (130, 2), (130, 2))

# Load Tokenizer

In [4]:
MODEL_NAME = "t5-base"
MAX_VOCAB = 20000

In [5]:
new_tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/t5-skinchat-training-1/outputs/simplet5-epoch-4-train-loss-1.4577-val-loss-0.8898/")

# Load Model

In [6]:
model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/t5-skinchat-training-1/outputs/simplet5-epoch-4-train-loss-1.4577-val-loss-0.8898/")

In [7]:
# instantiate
simplet5 = SimpleT5()

simplet5.model = model
simplet5.tokenizer = new_tokenizer

In [8]:
total_params = sum(p.numel() for p in simplet5.model.parameters())
print(f"Tổng số tham số của mô hình: {total_params}")
simplet5.model

Tổng số tham số của mô hình: 213589248


T5ForConditionalGeneration(
  (shared): Embedding(20000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

# Training model

In [9]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
simplet5.train(train_df=train_df,
               eval_df=valid_df,
               source_max_token_len=100,
               target_max_token_len=200,
               batch_size=24, 
               max_epochs=5, 
               use_gpu=True,
               precision=16
              )

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/core/lightning.py:2054: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
! (cd outputs; ls)

simplet5-epoch-0-train-loss-1.3586-val-loss-0.8571
simplet5-epoch-1-train-loss-1.2814-val-loss-0.8279
simplet5-epoch-2-train-loss-1.2171-val-loss-0.7905
simplet5-epoch-3-train-loss-1.158-val-loss-0.7691
simplet5-epoch-4-train-loss-1.1031-val-loss-0.7557
